<a href="https://colab.research.google.com/github/DavidbigC/Jupiter-bests-trader/blob/main/DC_Jupiter_data_action.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dune_client
!pip install dataclasses_json
!pip install ccxt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 18.0 MB/s eta 0:00:00


In [ ]:
from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import QueryBase
import pandas_datareader as pdr
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy
import ccxt
import json
import datetime


In [ ]:
dune_api_key = 'uczR4T2wYwCqKl73pUJQzOK1RHN0pIUY' # input your API key, to create one follow guide here https://docs.dune.com/api-reference/overview/authentication
query_id = 3948093
dune = DuneClient(
    api_key=dune_api_key,
    base_url="https://api.dune.com",
    request_timeout=(300)
)
query_result = dune.get_latest_result_dataframe(
    query=query_id
)


In [ ]:
phemex = ccxt.phemex({
    'enableRatelimit' : True,
    'apikey' : 'idonthaveakey',
    'secret' : 'idonthavesecret'

})
symbol = 'SOLUSDT'
timeframe = '2h'
limit = 1000
bars = phemex.fetch_ohlcv(symbol, timeframe = timeframe, limit = limit)
df = pd.DataFrame(bars, columns = ['timestamp','open','high','low','close','volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit = 'ms')

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=df['timestamp'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


In [ ]:
query_result['block_time'] = pd.to_datetime(query_result['block_time']).dt.tz_localize(None).dt.floor('h')
temp = query_result['block_time'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
for i in range(len(query_result)):
  if(int(temp[i][11:13]) % 2 == 1):
    temp2 = temp[i][0:10] + ' ' + str(int(temp[i][11:13])+1) + temp[i][13:19]
    if(str(int(temp[i][11:13])+1) == '24'):
      temp2 = temp[i][0:10] + ' ' + '00' + temp[i][13:19]
    temp2date = pd.to_datetime(temp2)
    temp[i] = temp2date

query_result['block_time'] = temp
query_result['block_time'] = pd.to_datetime(query_result['block_time'])
soltrade = query_result[query_result['token'] == 'SOL']
soltrade

<ipython-input-7-1cdcfc9705ae>:7: SettingWithCopyWarning:

modifications to a method of a datetimelike object are not supported and are discarded. Change values on the original.



,position_change,size,price_usd,pnls,direction,token,block_time,tot
0,increase,11350.983695,114.084769,<nil>,short,SOL,2024-01-02 12:00:00,<nil>
1,increase,110414.577444,113.797445,<nil>,short,SOL,2024-01-02 14:00:00,<nil>
2,decrease,-121765.561139,113.392369,461.925164,short,SOL,2024-01-02 14:00:00,461.925164
3,increase,113423.088325,110.740000,<nil>,short,SOL,2024-01-02 14:00:00,461.925164
4,decrease,-113423.088325,109.446921,1324.408647,short,SOL,2024-01-02 16:00:00,1786.333811
...,...,...,...,...,...,...,...,...
1750,decrease,-730248.214000,185.400000,3509.884768,long,SOL,2024-07-29 00:00:00,905633.1199280004
1751,increase,379429.702970,191.954286,<nil>,long,SOL,2024-07-29 12:00:00,905633.1199280004
1752,decrease,-379429.702970,192.500239,1079.167279,long,SOL,2024-07-29 12:00:00,906712.2872070004
1753,increase,359859.518225,189.480262,<nil>,long,SOL,2024-07-29 14:00:00,906712.2872070004


In [ ]:
soltrade = soltrade.rename({'block_time':'timestamp'}, axis=1)
fullist = pd.merge(df,soltrade, how="left",on='timestamp')
soltrade = soltrade[soltrade['timestamp'] >= '2024-05-01 00:00:00']
increase = soltrade[soltrade['position_change']=='increase']
decrease = soltrade[soltrade['position_change']=='decrease']
increase.reset_index(inplace=True)
decrease.reset_index(inplace=True)

In [ ]:
for i in range(len(increase)):
    temp = increase.loc[i,'price_usd']
    fig.add_annotation(x=increase.loc[i,'timestamp'], y=temp-5, text="▲", showarrow=False, font=dict(size=16, color='LightSeaGreen'))

for i in range(len(decrease)):
    temp = decrease.loc[i,'price_usd']
    fig.add_annotation(x=decrease.loc[i,'timestamp'], y=temp+5, text="▲", showarrow=False, font=dict(size=16, color='red'))

In [ ]:
fig.show()